In [ ]:
import datetime
import glob
import json
import os
import re
import time

# from siphon.catalog import TDSCatalog
from urllib.parse import urlparse

import ee
import geemap
import geopandas
import requests

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
shp_file = "../data/raw/region/IGM_MS_coast.shp"

In [ ]:
region = geopandas.GeoDataFrame.from_file(shp_file)

In [ ]:
region = region[region.shape_Area > 1].reset_index(drop=True)
region = region.to_crs("EPSG:4326")

In [ ]:
geojson_file = "../data/raw/region/region.geojson"

In [ ]:
region.to_file(geojson_file, driver="GeoJSON")

In [ ]:
feature = geemap.geojson_to_ee(geojson_file)

In [ ]:
feature = ee.FeatureCollection("TIGER/2018/States").filter(
    ee.Filter.eq("NAME", "Mississippi")
)

In [ ]:
start_date = "2002-04-01"
end_date = datetime.datetime.now().strftime("%Y-%m-%d")

In [ ]:
months = ee.List.sequence(1, 12)
years = ee.List.sequence(2002, 2022)

In [ ]:
terra_climate_bands = ["aet", "def", "pdsi", "pet", "pr", "srad", "ro", "soil", "swe"]

In [ ]:
terra_climate_collection = (
    ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")
    # .filterBounds(feature)
    .filter(ee.Filter.date(start_date, end_date))
    .select(terra_climate_bands)
    .map(lambda image: image.clipToBoundsAndScale(feature))
)

In [ ]:
terra_climate_out_dir = os.path.join("../data/raw/terra_climate")

In [ ]:
geemap.ee_export_image_collection(terra_climate_collection, terra_climate_out_dir)

In [ ]:
# Download GRACE Data

In [ ]:
earthdata_username = ""
earthdata_pass = ""
SESSION = requests.session()
AUTH = (earthdata_username, earthdata_pass)

In [ ]:
def download_grace_catalog(grace_url, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    grace_catalog = TDSCatalog(grace_url)

    for name, ds in grace_catalog.datasets.items():
        if "GRCTellus.JPL" in name and name.endswith("nc"):
            file_url = ds.access_urls["dap"]
            file_name = file_url.split("/")[-1]
            end_date = re.search(r"200204_\d{6}", file_name).group(0).split("_")[1]
            end_date_obj = datetime.datetime.strptime(end_date, "%Y%m")
            print(end_date_obj)
            file_list = glob.glob(os.path.join(output_dir, "*200204*"))
            parts = urlparse(file_url)
            updated_url = parts._replace(path="opendap/hyrax" + parts.path).geturl()
            output_path = os.path.join(output_dir, file_name)
            print(file_list)
            if len(file_list) == 0:
                print("downloading at empty")
                download_catalog_url(updated_url, output_path)
            else:
                print("delete the  previous file")
                old_file = file_list[0].split("/")[-1]
                prev_end_date = (
                    re.search(r"200204_\d{6}", old_file).group(0).split("_")[1]
                )
                prev_end_date_obj = datetime.datetime.strptime(prev_end_date, "%Y%m")
                if end_date_obj > prev_end_date_obj:
                    print("deleting the previous file")
                    os.remove(file_list[0])
                    download_catalog_url(updated_url, output_path)
        if "SCALE_FACTOR" in name:
            output_file = "scale_factors.nc"
            output_path = os.path.join(output_dir, output_file)
            if not os.path.exists(output_path):
                file_url = ds.access_urls["dap"]
                parts = urlparse(file_url)
                updated_url = parts._replace(path="opendap/hyrax" + parts.path).geturl()
                download_catalog_url(updated_url, output_path)

    return True

In [ ]:
def download_catalog_url(final_url, output_path):
    with requests.Session() as session:
        r1 = session.request("get", final_url)
        r = session.get(r1.url, auth=AUTH)
        if r.status_code == 200:
            with open(output_path, "wb") as f:
                f.write(r.content)
                return True

In [ ]:
mascon_dir = "../data/raw/mascon"

In [ ]:
download_grace_catalog(
    "https://opendap.jpl.nasa.gov/opendap/hyrax/"
    "allData/tellus/L3/mascon/RL06/JPL/v02/CRI/"
    "netcdf/catalog.xml",
    mascon_dir,
)

In [ ]:
# Download chirps from https://climateserv.servirglobal.net/map

In [ ]:
chirps_ic = (
    ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
    .select("precipitation")
    .filterDate(start_date, end_date)
    .map(lambda image: image.clipToBoundsAndScale(feature))
)

In [ ]:
def function1(y):
    def function2(m):
        return (
            chirps_ic.filter(ee.Filter.calendarRange(y, y, "year"))
            .filter(ee.Filter.calendarRange(m, m, "month"))
            .mean()
            .set("month", m)
            .set("year", y)
        )

    return months.map(function2)

In [ ]:
chirps_monthly = ee.ImageCollection.fromImages(years.map(function1).flatten())

In [ ]:
chirps_out_dir = os.path.join("../data/raw/chirps")